In [ ]:
# Load Required Packages
import sklearn
import statistics
import scipy as sc
import numpy as np
import pandas as pd
import random
import cvxpy as cp
import matplotlib.pyplot as plt
import multiprocessing as mp
from multiprocessing import Pool
import time
import re
import sys
import xgboost as xgb
import gc
import mosek

from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from pmlb import fetch_data
from pmlb import classification_dataset_names, regression_dataset_names

from sklearn.datasets import load_boston
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer
pd.set_option('display.max_columns', None)
sys.path.append('../')

from IncrementalDepthBagging.IncrementalDepthBagging import IncrementalDepthBaggingClassifier_fit
from IncrementalDepthBagging.IncrementalDepthBagging import IncrementalDepthBaggingClassifier_predict_proba

In [ ]:
def IncrementalDepthBagBoostClassifier_predict(xTest,log_odds_init,tree_list):
    res = log_odds_predict(xTest,log_odds_init,tree_list)
    return np.exp(res)/(1+np.exp(res))

def log_odds_predict(xTest,log_odds_init,tree_list):
    res = []
    for i in tree_list:
        depth = i.max_depth
        pred = i.predict(xTest)
        res.append([depth,pred])
    res = pd.DataFrame(res,columns = ['depth','pred'])
    res = res.groupby('depth')['pred'].apply(np.mean).reset_index()
    res = np.sum(res['pred'].to_numpy()) + log_odds_init
    return res


def converge_test(sequence, threshold,length):
    diff = np.diff(sequence)
    if len(diff) < (length+1):
        return False
    else:
        return (max(np.abs(diff[-length:])) < threshold)
    
def check_OOB_convergence(OOB_error_list):
    if OOB_error_list[-1] <= 0:
        return True
    elif (len(OOB_error_list) < 4):
        return False
    elif all([x < 10**-4 for x in OOB_error_list[-3:]]):
        return True
    else:
        return False

In [ ]:
def IncrementalDepthBagBoostClassifier_OOB_EarlyStop(xTrain,yTrain, threshold,tail):
    yTrain = pd.Series(yTrain)
    yTrain.index = xTrain.index
    #initialization
    log_odds = np.log(sum(yTrain)/(len(yTrain)- sum(yTrain)))
    prob = np.exp(log_odds)/(1+np.exp(log_odds))
    residual = yTrain - prob

    train = xTrain.copy()
    train['yTrain'] = list(residual)
    features = xTrain.columns
    pred_train = np.zeros(len(residual))
    tree_list = []
    
    OOB_error_list = []
    OOB_converged = False
    depth = 1
    while OOB_converged == False:
    
        early_stop_pred = []
        early_stop_train_err = []
        converged = False
        OOB_matrix = []
        tree_list1 = []

        if len(tree_list) > 0:
            current_pred = log_odds_predict(xTrain,log_odds,tree_list)
            xTrain['current_pred'] = current_pred
            current_pred = xTrain['current_pred']
            xTrain.drop('current_pred',axis = 1,inplace = True)
        else:
            xTrain['current_pred'] = log_odds
            current_pred = xTrain['current_pred']
            xTrain.drop('current_pred',axis = 1,inplace = True)
        
        
        while converged == False:
            
            train1 = train.sample(n = len(train), replace = True)
            OOB = train[~train.index.isin(train1.drop_duplicates().index.values)].index.values
            OOB_row = np.repeat(False,len(xTrain))
            OOB_row[OOB] = True
            OOB_matrix.append(OOB_row)        
            yTrain1 = train1['yTrain']
            xTrain1 = train1[features]
            tree = DecisionTreeRegressor(max_depth = depth)
            tree.fit(xTrain1,yTrain1)
            tree_list.append(tree)
            tree_list1.append(tree)
            pred = tree.predict(xTrain[features])
            early_stop_pred.append(pred)
            
            temp_pred = current_pred + (np.mean(early_stop_pred,axis = 0))
            temp_prob = np.exp(temp_pred)/(1+np.exp(temp_pred))
        
            early_stop_train_err.append(sklearn.metrics.roc_auc_score(yTrain,temp_prob))
            #early_stop_train_err.append(sklearn.metrics.mean_squared_error(train['yTrain'],(np.mean(early_stop_pred,axis = 0))))
            converged = converge_test(early_stop_train_err,threshold,tail)
            pred_train = pred_train + np.mean(early_stop_pred,axis = 0)
            if converged == False:
                pred_train = pred_train - np.mean(early_stop_pred,axis = 0)
                
        ### compute OOB
        indicators = pd.DataFrame(OOB_matrix).transpose()
        OOB_pred_list = []
        #yTrain2 = train['yTrain'].copy()
        yTrain2 = yTrain.copy()
        for i,row in xTrain.iterrows():
            row = row.to_frame().transpose()
            temp_series = indicators.iloc[i]
            OOB_trees = list(temp_series[temp_series].index.values)
            OOB_tree_list = list(np.array(tree_list1)[OOB_trees])
        
            if len(OOB_tree_list) > 0:
                OOB_pred = []
                for tree_temp in OOB_tree_list:
                    OOB_pred.append(tree_temp.predict(row)[0])
                OOB_pred_list.append(np.mean(OOB_pred))
            else:
                yTrain2 = yTrain2.drop(i)
                current_pred = current_pred.drop(i)
        
        next_pred = np.array(current_pred) + np.array(OOB_pred_list)
        
        current_prob =  np.exp(current_pred)/(1+np.exp(current_pred))
        next_prob =  np.exp(next_pred)/(1+np.exp(next_pred))
        
        current_err = 1 - sklearn.metrics.roc_auc_score(yTrain2,current_prob)
        next_err = 1 - sklearn.metrics.roc_auc_score(yTrain2,next_prob)
        
        print(1-current_err,1-next_err, depth)
        OOB_error_list.append(current_err-next_err)
        
        all_pred = log_odds_predict(xTrain,log_odds,tree_list)
        all_prob = np.exp(all_pred)/(1+np.exp(all_pred))
        train['yTrain'] = yTrain-all_prob
        OOB_converged = check_OOB_convergence(OOB_error_list)
        depth = depth + 1
        
    return tree_list

In [ ]:
def OptimizationStepClassification(xTrain,yTrain,tree_list,lambd, optimization_type = 'penalized'):
    pred = []
    ind = []
    
    yTrain = pd.Series(yTrain)
    yTrain.index = xTrain.index
    #initialization
    log_odds = np.log(sum(yTrain)/(len(yTrain)- sum(yTrain)))
    
    for tree in tree_list:
        pred.append(tree.predict(xTrain))
        ind.append([int(x > 0) for x in tree.feature_importances_])  

    pred = np.transpose(pred)
    ind = np.transpose(ind)
    
    w = cp.Variable(len(tree_list),nonneg=True)
    constraints = []
    
    
    if optimization_type == 'penalized':
        loss = -cp.sum( cp.multiply(yTrain, pred@ w ) - cp.logistic(pred @ w) )
        objective = (1/len(yTrain))*loss + lambd*cp.norm(cp.matmul(ind,w),1)


    if optimization_type == 'constrained':
        objective = -cp.sum(cp.multiply(yTrain, pred@ w) - cp.logistic(pred @ w))
        constraints = [cp.norm(cp.matmul(ind,w),1)<= lambd]

    prob = cp.Problem(cp.Minimize(objective),constraints)
    prob.solve(solver = cp.MOSEK,mosek_params = {mosek.dparam.optimizer_max_time: 10000.0} )
    weights = np.asarray(w.value)
    weights[np.abs(weights) < 10**-3] = 0 
    return weights

def ControlBurnClassifier_predict(xTest, log_odds_init, tree_list, weights):
    res = []
    for i in range(0,len(tree_list)):
        res.append(weights[i]*tree_list[i].predict(xTest))
    pred = log_odds_init + np.sum(res,axis = 0)
    prob = np.exp(pred)/(1+np.exp(pred))
    return prob

def ControlBurnClassifier_select_features(xTest, tree_list,weights):
    imp = []
    for i in range(0,len(weights)):
        imp.append(weights[i]*tree_list[i].feature_importances_)
    imp1 = np.sum(imp, axis = 0)
    return imp1


In [ ]:
def baseline_classifiers(xTrain,yTrain,xTest,yTest, num_features, num_trees):
    model = RandomForestClassifier(n_estimators = num_trees)
    rf = model.fit(xTrain,yTrain)
    imp = pd.DataFrame(np.column_stack((xTrain.columns,rf.feature_importances_)),columns = ['features','scores'])
    imp = imp.sort_values('scores',ascending = False)
    to_use = imp.head(num_features)['features'].values
    rf1 = model.fit(xTrain[to_use],yTrain)
    pred = rf1.predict_proba(xTest[to_use])[:,1]
    return sklearn.metrics.roc_auc_score(yTest,pred)

def evaluate_classification_experiment(xTrain,yTrain,xTest,yTest,lambd,tree_list,form):
    weights = OptimizationStepClassification(xTrain, yTrain,tree_list, lambd,form)
    imp1 = ControlBurnClassifier_select_features(xTest, tree_list,weights)
    
    if sum(imp1>0) == 0:
        return [.5,5,0]
    
    rf = RandomForestClassifier(n_estimators = 100).fit(xTrain[xTrain.columns[imp1 > 0]],yTrain)
    pred_polish = rf.predict_proba(xTest[xTrain.columns[imp1 > 0]])[:,1]
    acc_polish = sklearn.metrics.roc_auc_score(yTest,pred_polish)

    importances = pd.DataFrame(np.column_stack((xTrain.columns,imp1)),columns = ['features','scores'])
    importances = importances.sort_values('scores',ascending = False)
    num_features = np.sum(importances['scores'] != 0)
    
    acc_base = baseline_classifiers(xTrain,yTrain,xTest,yTest, np.sum(imp1>0), len(tree_list))

    return [acc_polish,acc_base,num_features]

def classification_bisection_lambd(xTrain,yTrain,xTest,yTest,tree_list,form,lambd,count_limit,feature_to_find):
    counter = 0
    to_find = 0
    total_results = []
    while to_find <= feature_to_find:
        results = evaluate_classification_experiment(xTrain,yTrain,xTest,yTest,lambd,tree_list,form)
        nfeat = results[2]
        print(to_find,nfeat,lambd)

        if nfeat == to_find:
            to_find = to_find + 1

        elif counter > count_limit:
            to_find = to_find + 1
            counter = 0

        elif nfeat < to_find:
            lambd = lambd/2

        elif nfeat > to_find:
            lambd = lambd + lambd/2

        counter = counter + 1

        total_results.append(results)
    return total_results

def full_model(X,y):
    kf = KFold(n_splits=5)
    kf.get_n_splits(X)
    results = []
    for train_index, test_index in kf.split(X):
        xTrain, xTest = X.iloc[train_index], X.iloc[test_index]
        yTrain, yTest = y.iloc[train_index], y.iloc[test_index]
        features = xTrain.columns
        xTrain = preprocessing.scale(xTrain)
        xTrain = pd.DataFrame(xTrain,columns = features)
        xTest = preprocessing.scale(xTest)
        xTest = pd.DataFrame(xTest,columns = features)
        yTest = pd.Series(yTest)
        yTrain = pd.Series(yTrain)
        yTest.index = xTest.index
        yTrain.index = xTrain.index
        pred = RandomForestClassifier(n_estimators = 1000, max_features = 'sqrt').fit(xTrain,yTrain).predict_proba(xTest)[:,1]
        results.append(sklearn.metrics.roc_auc_score(yTest,pred))
    return results


In [ ]:
def load_audit():
    audit_risk = pd.read_csv("data/audit_risk.csv")
    trial = pd.read_csv("data/trial.csv")
    trial.columns = ['Sector_score','LOCATION_ID', 'PARA_A', 'Score_A', 'PARA_B',
           'Score_B',  'TOTAL', 'numbers', 'Marks',
           'Money_Value', 'MONEY_Marks', 'District',
           'Loss', 'LOSS_SCORE', 'History', 'History_score', 'Score', 'Risk_trial' ]
    trial['Score_A'] = trial['Score_A']/10
    trial['Score_B'] = trial['Score_B']/10
    merged_df = pd.merge(audit_risk, trial, how='outer', on = ['History', 'LOCATION_ID', 'Money_Value', 'PARA_A', 'PARA_B',
           'Score', 'Score_A', 'Score_B', 'Sector_score', 'TOTAL', 'numbers'])

    df = merged_df.drop(['Risk_trial'], axis = 1)
    df['Money_Value'] = df['Money_Value'].fillna(df['Money_Value'].median())
    df = df.drop(['Detection_Risk', 'Risk_F'], axis = 1) 
    df = df[(df.LOCATION_ID != 'LOHARU')]
    df = df[(df.LOCATION_ID != 'NUH')]
    df = df[(df.LOCATION_ID != 'SAFIDON')]
    df = df.astype(float)
    df = df.drop_duplicates(keep = 'first')
    class_df = df.drop(["Audit_Risk",'Inherent_Risk','Score','TOTAL'], axis = 1)
    y = class_df["Risk"]    
    X = class_df.drop(["Risk"], axis = 1)

    return X,y


In [ ]:
## Load Data

# ###########################################################
# #PMLB

# dataset = 'twonorm'

# data = fetch_data(dataset)
# data = data.sample(len(data))
# y = data['target']
# X = data.drop('target',axis = 1)


# ###########################################################
# #OpenML

# dataset = 'lung-cancer'
# from sklearn.datasets import fetch_openml
# data = fetch_openml(name=dataset)
# #X = pd.DataFrame(data.data,columns = data.feature_names)
# X = pd.DataFrame.sparse.from_spmatrix(data.data, columns = data.feature_names).sparse.to_dense()
# y = pd.Series(data.target).map({-1:0,1:1})



###########################################################
print(dataset)
print(len(X),len(X.columns))
kf = KFold(n_splits=5)
kf.get_n_splits(X)

final_result_bagboost = pd.DataFrame(None)
final_result_bag = pd.DataFrame(None)
folds_data = []

for train_index, test_index in kf.split(X):
    xTrain, xTest = X.iloc[train_index], X.iloc[test_index]
    yTrain, yTest = y.iloc[train_index], y.iloc[test_index]
    
    features = xTrain.columns
    xTrain = preprocessing.scale(xTrain,with_mean=False)
    xTrain = pd.DataFrame(xTrain,columns = features)
    xTest = preprocessing.scale(xTest,with_mean=False)
    xTest = pd.DataFrame(xTest,columns = features)
    yTest = pd.Series(yTest)
    yTrain = pd.Series(yTrain)
    yTest.index = xTest.index
    yTrain.index = xTrain.index
    
    folds_data.append([xTrain,xTest,yTrain,yTest])

In [ ]:
%%time
import ray
import time

# Start Ray
ray.shutdown()
ray.init()

@ray.remote
def parallel_wrapper(arg):
    xTrain = arg[0]
    xTest = arg[1]
    yTrain = arg[2]
    yTest = arg[3]
    
    features_to_find = min(len(X.columns),10)
    
    tree_list = IncrementalDepthBagBoostClassifier_OOB_EarlyStop(xTrain,yTrain,10**-3,5)
    res = classification_bisection_lambd(xTrain,yTrain,xTest,yTest,tree_list,'penalized',10,8,features_to_find)
    res = pd.DataFrame(res, columns = ['polish','base','nonzero'])
    
    return res

result_ids = []
for i in folds_data:
    result_ids.append(parallel_wrapper.remote(i))
    

results = ray.get(result_ids)  
ray.shutdown()

final_result_bagboost = pd.DataFrame(None)

for r in results:
    final_result_bagboost = final_result_bagboost.append(r)

    
final_result_agg = final_result_bagboost.groupby('nonzero').agg(['mean','std']).reset_index()
final_result_agg = final_result_agg[final_result_agg['nonzero'] != 0]

In [ ]:
full = full_model(X,y)
final_result_agg['full_mean'] = np.mean(full)
final_result_agg['full_std']= np.std(full)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(final_result_agg['nonzero'],final_result_agg['base']['mean'],label = 'baseline', color = 'blue')
plt.scatter(final_result_agg['nonzero'],final_result_agg['base']['mean'],color = 'blue')
plt.errorbar(final_result_agg['nonzero'],final_result_agg['base']['mean'],yerr = final_result_agg['base']['std'],color = 'blue')


plt.plot(final_result_agg['nonzero'],final_result_agg['polish']['mean'],label = 'polished',color = 'green')
plt.scatter(final_result_agg['nonzero'],final_result_agg['polish']['mean'], color = 'green')
plt.errorbar(final_result_agg['nonzero'],final_result_agg['polish']['mean'],yerr = final_result_agg['polish']['std'], color = 'green')
plt.xlim(0,10)
#plt.axhline(final_result_agg['full_mean'].values[0],label = 'full RF model')
plt.legend()
plt.title(dataset+ ' bagboost')
plt.ylabel('ROC AUC')
plt.xlabel('Features Selected')


In [ ]:
final_result_agg.to_csv('../Results/'+dataset+'_bagboost_5fold.csv',index= False)

In [ ]:
# Plotting function

In [ ]:
import matplotlib.pyplot as plt
dataset = 'pima'

final_result_agg = pd.read_csv('bagboostresults/'+dataset+'_bagboost_5fold.csv')
final_result_agg.drop(0,axis = 0,inplace = True)
final_result_agg.columns = ['nonzero','polish_mean','polish_std','base_mean','base_std','full_mean','full_std']
final_result_agg = final_result_agg.astype(float)
final_result_agg = final_result_agg[final_result_agg['nonzero'] < 10]
                                    
                                    
                                    
plt.plot(final_result_agg['nonzero'],final_result_agg['base_mean'],label = 'baseline', color = 'blue')
plt.scatter(final_result_agg['nonzero'],final_result_agg['base_mean'],color = 'blue')
plt.errorbar(final_result_agg['nonzero'],final_result_agg['base_mean'],yerr = final_result_agg['base_std'],color = 'blue')


plt.plot(final_result_agg['nonzero'],final_result_agg['polish_mean'],label = 'polished',color = 'green')
plt.scatter(final_result_agg['nonzero'],final_result_agg['polish_mean'], color = 'green')
plt.errorbar(final_result_agg['nonzero'],final_result_agg['polish_mean'],yerr = final_result_agg['polish_std'], color = 'green')

plt.axhline(final_result_agg['full_mean'].values[0],label = 'full RF model')
plt.legend()
plt.title(dataset+ ' bagboost')
plt.ylabel('ROC AUC')
plt.xlabel('Features Selected')
